# <font color="#067790">Projet 4 : Analysez les ventes de votre entreprise</font>

### Dans ce projet, il est prévu d'analyser les KPI liées aux performances commerciales et financières de la société.

Ce travail est découpé en 2 grandes parties : l'analyse marketing des variables de cet échantillon et la recherche de corrélations entre elles.<br/>


Ce premier notebook traite du nettoyage des données et du formatage en dataframe distincts.

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Import des data 

In [14]:
#Import des données des tables
dfc = pd.read_csv("data/customers.csv")
dft = pd.read_csv("data/transactions.csv")
dfp = pd.read_csv("data/products.csv")
print("La taille du dataframe customers ",dfc.shape)
print("La taille du dataframe transactions",dft.shape)
print("La taille du dataframe produits",dfp.shape)

La taille du dataframe customers  (8623, 3)
La taille du dataframe transactions (337016, 4)
La taille du dataframe produits (3287, 3)


In [15]:
# Fusion en un seul df
df = pd.merge(dft, dfp, on=['id_prod'], how ='left')
df = pd.merge(df, dfc, on=['client_id'], how ='left')
df.head()

,id_prod,date,session_id,client_id,price,categ,sex,birth
0,0_1483,2021-04-10 18:37:28.723910,s_18746,c_4450,4.99,0.0,f,1977
1,2_226,2022-02-03 01:55:53.276402,s_159142,c_277,65.75,2.0,f,2000
2,1_374,2021-09-23 15:13:46.938559,s_94290,c_4270,10.71,1.0,f,1979
3,0_2186,2021-10-17 03:27:18.783634,s_105936,c_4597,4.20,0.0,m,1963
4,0_1351,2021-07-17 20:34:25.800563,s_63642,c_1242,8.99,0.0,f,1980


## <font color="#067790">Partie 1 : Nettoyage</font>


### 1. Traitement des valeurs manquantes

In [16]:
# Créer une nouvelle fonction qui détermine si la valeur en paramètre est manquante:
def num_missing(x):
    return sum(x.isnull())

# On applique cette fonction pour chaque colonne:
print("Valeurs manquantes par colonne:")
print(df.apply(num_missing, axis=0)) #axis=0 définit que la fonction sera bien appliquée sur chaque colonne

# Puis application pour chaque ligne:
print("\nValeurs manquantes par ligne:")
print(df.apply(num_missing, axis=1).head()) #axis=1 définit que la fonction sera bien appliquée sur chaque ligne

Valeurs manquantes par colonne:
id_prod         0
date            0
session_id      0
client_id       0
price         103
categ         103
sex             0
birth           0
dtype: int64

Valeurs manquantes par ligne:
0    0
1    0
2    0
3    0
4    0
dtype: int64


In [17]:
# Recherche des 103 valeurs manquantes
df.loc[df.price.isnull() & df.categ.isnull()]

,id_prod,date,session_id,client_id,price,categ,sex,birth
6235,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533,NaN,NaN,m,1972
10802,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954,NaN,NaN,m,1973
14051,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120,NaN,NaN,f,1975
17486,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964,NaN,NaN,f,1982
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580,NaN,NaN,m,1988
...,...,...,...,...,...,...,...,...
322710,0_2245,2021-04-06 19:59:19.462288,s_16936,c_4167,NaN,NaN,f,1979
329417,0_2245,2021-03-30 23:29:02.347672,s_13738,c_7790,NaN,NaN,f,1983
330490,0_2245,2021-12-03 14:14:40.444177,s_128815,c_6189,NaN,NaN,f,1984
335531,0_2245,2021-04-27 18:58:47.703374,s_26624,c_1595,NaN,NaN,f,1973


In [18]:
# Remplacement des valeurs manquantes en cat 0
df.categ.fillna(0, inplace=True)

# Remplacement des valeurs manquantes par la moyennes de prix de cat 0
a = round(df.loc[df.categ==0].price.mean(),2)
df.price.fillna(a,inplace =True)

In [19]:
# Vérification du traitement
df.loc[df.id_prod == '0_2245'].head()

,id_prod,date,session_id,client_id,price,categ,sex,birth
6235,0_2245,2021-06-17 03:03:12.668129,s_49705,c_1533,10.64,0.0,m,1972
10802,0_2245,2021-06-16 05:53:01.627491,s_49323,c_7954,10.64,0.0,m,1973
14051,0_2245,2021-11-24 17:35:59.911427,s_124474,c_5120,10.64,0.0,f,1975
17486,0_2245,2022-02-28 18:08:49.875709,s_172304,c_4964,10.64,0.0,f,1982
21078,0_2245,2021-03-01 00:09:29.301897,s_3,c_580,10.64,0.0,m,1988


In [20]:
# Vérifier la fonction après le retraitement
def num_missing(x):
    return sum(x.isnull())

# On applique cette fonction pour chaque colonne:
print("Valeurs manquantes par colonne:")
print(df.apply(num_missing, axis=0)) #axis=0 définit que la fonction sera bien appliquée sur chaque colonne

# Puis application pour chaque ligne:
print("\nValeurs manquantes par ligne:")
print(df.apply(num_missing, axis=1).head()) #axis=1 définit que la fonction sera bien appliquée sur chaque ligne

Valeurs manquantes par colonne:
id_prod       0
date          0
session_id    0
client_id     0
price         0
categ         0
sex           0
birth         0
dtype: int64

Valeurs manquantes par ligne:
0    0
1    0
2    0
3    0
4    0
dtype: int64


In [10]:
# Ajout du produit manquant au catalogue produit 
dfp.loc[3287] = ["0_2245",10.64, 0]

### 2. Traitement des valeurs incohérentes

In [21]:
# Repérage et suppresion des prix négatifs car coorespond aux clients tests
dfp.loc[dfp.price <0]
df = df.loc[df.price>0]
dfp = dfp.drop([731])

In [22]:
# Réperage des données test 
dfc.loc[(dfc.client_id =='ct_0') | (dfc.client_id =='ct_1')]
df.loc[df.client_id =='ct_1']

,id_prod,date,session_id,client_id,price,categ,sex,birth


In [23]:
# Suppresion de ces 2 données au catalogue client 
dfc = dfc.drop([2735])
dfc = dfc.drop([8494])

In [39]:
dfp.to_csv("data/products_updated.csv", index = False)
df.to_csv("data/transactions_updated.csv", index = False)

## Construction des dataframes

In [25]:
# Ajout de l'age au dataframes DFC et DF
dfc['age']= 2022 - dfc.birth
df['age']= 2022 - df.birth

In [26]:
# Ajout de la date en mois et en année
df.date = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [27]:
# Construction de DFS : nombre de session par client + panier session par client
a1 = df.client_id.groupby(df.session_id).count().sort_values(ascending = False).to_frame()
b1 = df.price.groupby(df.session_id).sum().sort_values(ascending = False).to_frame()
b1.reset_index()
dfs = pd.merge(a1,b1,on='session_id')
dfs.reset_index()
dfs.head()

,client_id,price
session_id,,
s_118668,14,264.99
s_96857,13,277.31
s_21005,13,162.20
s_168560,12,182.24
s_93892,12,130.58


In [28]:
# Vérification de la construction de DFS
df.loc[df.session_id =='s_118668']

,id_prod,date,session_id,client_id,price,categ,sex,birth,age,year,month,day
32767,0_1108,2021-11-12 20:20:33.659396,s_118668,c_5995,17.45,0.0,f,1992,30,2021,11,12
37741,0_1334,2021-11-12 20:08:40.783542,s_118668,c_5995,17.74,0.0,f,1992,30,2021,11,12
56278,0_1400,2021-11-12 21:24:36.491177,s_118668,c_5995,5.15,0.0,f,1992,30,2021,11,12
107722,2_209,2021-11-12 20:00:00.956739,s_118668,c_5995,69.99,2.0,f,1992,30,2021,11,12
111473,1_81,2021-11-12 20:24:02.266858,s_118668,c_5995,33.99,1.0,f,1992,30,2021,11,12
120480,0_1430,2021-11-12 20:36:39.252082,s_118668,c_5995,16.47,0.0,f,1992,30,2021,11,12
133028,0_1488,2021-11-12 20:23:50.319430,s_118668,c_5995,4.60,0.0,f,1992,30,2021,11,12
156789,0_1089,2021-11-12 20:02:54.200624,s_118668,c_5995,3.99,0.0,f,1992,30,2021,11,12
158735,1_498,2021-11-12 20:18:11.950251,s_118668,c_5995,23.37,1.0,f,1992,30,2021,11,12
167546,0_1564,2021-11-12 20:09:17.901543,s_118668,c_5995,11.12,0.0,f,1992,30,2021,11,12


In [29]:
# DFS : renommage colonnes et merge avec données utiles
df1 = df.loc[:,['session_id','client_id','year','month','age','sex']]
dfs = pd.merge(dfs,df1,on='session_id').drop_duplicates()
dfs.head()

,session_id,client_id_x,price,client_id_y,year,month,age,sex
0,s_118668,14,264.99,c_5995,2021,11,30,f
14,s_96857,13,277.31,c_5523,2021,9,52,f
27,s_21005,13,162.20,c_8303,2021,4,32,m
40,s_168560,12,182.24,c_7855,2022,2,32,f
52,s_93892,12,130.58,c_8127,2021,9,34,f


In [30]:
dfs.rename(columns={'client_id': 'sales_count', 'price': 'sales_sum','client_id_y': 'client_id'}, inplace=True)

In [31]:
dfs.columns = ['session_id','sales_count','sales_sum','client_id','year','month','age','sex']

In [32]:
dfs.head()

,session_id,sales_count,sales_sum,client_id,year,month,age,sex
0,s_118668,14,264.99,c_5995,2021,11,30,f
14,s_96857,13,277.31,c_5523,2021,9,52,f
27,s_21005,13,162.20,c_8303,2021,4,32,m
40,s_168560,12,182.24,c_7855,2022,2,32,f
52,s_93892,12,130.58,c_8127,2021,9,34,f


### Export des dataframes cleanés

In [38]:
# Export des dataframes cleaned :
dfp.to_csv("data/products_cleaned.csv", index = False)
dfc.to_csv("data/customers_cleaned.csv", index = False)
df.to_csv("data/transactions_cleaned.csv", index = False)
dfs.to_csv('data/sessions_cleaned.csv',index = False)

## M3 : Exploitation corrélation sans outliers

#### Valeurs extremes CLIENT ANORMAUX : comportement B to B

In [34]:
# DF sans les 4 premiers clients : c_1609 / c_4958 / c_6714 / c_3454
df.groupby('client_id').price.sum().sort_values(ascending = False).head()

client_id
c_1609    162007.34
c_4958    144257.21
c_6714     73218.62
c_3454     54464.20
c_7959      2564.25
Name: price, dtype: float64

In [35]:
dfs.head()

,session_id,sales_count,sales_sum,client_id,year,month,age,sex
0,s_118668,14,264.99,c_5995,2021,11,30,f
14,s_96857,13,277.31,c_5523,2021,9,52,f
27,s_21005,13,162.20,c_8303,2021,4,32,m
40,s_168560,12,182.24,c_7855,2022,2,32,f
52,s_93892,12,130.58,c_8127,2021,9,34,f


In [36]:
dfs.loc[(dfs.client_id != 'c_1609') & (dfs.client_id != 'c_4958') & (dfs.client_id != 'c_6714') & (dfs.client_id != 'c_3454')]

,session_id,sales_count,sales_sum,client_id,year,month,age,sex
0,s_118668,14,264.99,c_5995,2021,11,30,f
14,s_96857,13,277.31,c_5523,2021,9,52,f
27,s_21005,13,162.20,c_8303,2021,4,32,m
40,s_168560,12,182.24,c_7855,2022,2,32,f
52,s_93892,12,130.58,c_8127,2021,9,34,f
...,...,...,...,...,...,...,...,...
336811,s_172326,1,25.79,c_2801,2022,2,58,m
336812,s_78371,1,26.64,c_91,2021,8,51,m
336813,s_172329,1,25.99,c_4770,2022,2,52,m
336814,s_78370,1,18.99,c_3060,2021,8,51,m


In [37]:
# Suppression de ces 4 clients dans les dataframe : DF, DFC et DFS
dfs_SO = dfs.loc[(dfs.client_id != 'c_1609') & (dfs.client_id != 'c_4958') & (dfs.client_id != 'c_6714') & (dfs.client_id != 'c_3454')]
dfc_SO = dfc.loc[(dfc.client_id != 'c_1609') & (dfc.client_id != 'c_4958') & (dfc.client_id != 'c_6714') & (dfc.client_id != 'c_3454')]
df_SO = df.loc[(df.client_id != 'c_1609') & (df.client_id != 'c_4958') & (df.client_id != 'c_6714') & (df.client_id != 'c_3454')]